In [5]:
import numpy as np, torch

In [6]:
train, test = (np.load(f) for f in ("../data/task1_train.npz",
                                    "../data/task1_test.npz"))
xtr, ytr, xte, yte = (
  torch.from_numpy(arr)
  for arr in (train["x"], train["y"], test["x"], test["y"])
)